<a href="https://colab.research.google.com/github/thomasD75/hello-world/blob/master/s3dis_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stanford 3D Indoor Scene Dataset (S3DIS)**

This notebook explores the [S3DIS](https://svl.stanford.edu/assets/papers/3D_Semantic_Parsing.pdf) dataset using [torch_geometric](https://pytorch-geometric.readthedocs.io/en/latest/index.html)

Install and import libraries

In [1]:
!pip install open3d>=0.16.0 # must be at least 0.16.0

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.9.0+cpu
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install open3d

In [ ]:
from IPython.display import clear_output
import numpy as np
import open3d as o3
# from open3d import JVisualizer # For Colab Visualization
# from open3d.web_visualizer import draw # for non Colab

import matplotlib.pyplot as plt
%matplotlib inline

Get Dataset

In [ ]:
import torch_geometric
from torch_geometric.datasets import S3DIS

s3dis_train = S3DIS(root='temp', test_area=6, train=True)

In [ ]:
s3dis_test = S3DIS(root='temp', test_area=6, train=False)

# split into valid and test sets
s3dis_valid = s3dis_test[:2000]
s3dis_test = s3dis_test[2000:]

In [ ]:
print(f'Number of train samples: {len(s3dis_train)}')
print(f'Number of validation samples: {len(s3dis_valid)}')
print(f'Number of test samples: {len(s3dis_test)}')
s3dis_train[0]

The 'x' portion of the data set contains 0-1 normalized color and position values that can be fed into a model. The 'y' portion of the dataset contains truth labels that can be found in the original dataset. (TRUTH LABEL MAPPING TBD). The pos contains raw 3D positions and is similar to shapenet.


Access normalized color x[:, :3], access normalized position x[:, 3:]

In [ ]:
s3dis_train[0].x.max(axis=0)

In [ ]:
s3dis_train[0].pos

In [ ]:
np.unique(s3dis_train[332].y)

## Display some stuff

First we will get a color mapping for the truth labels

In [ ]:
color_map = {
    0  : (255, 0, 0),
    1  : (0, 255, 0),
    2  : (0, 0, 255),
    3  : (255, 0, 255),
    4  : (0, 255, 255),
    5  : (255, 255, 0),
    6  : (128, 0, 128),
    7  : (128, 128, 0),
    8  : (0, 128, 128),
    9  : (128, 0, 255),
    10 : (255, 128, 0),
    11 : (0, 128, 255),
    12 : (128, 255, 128)
}

map_colors = lambda x : color_map[x]
v_map_colors = np.vectorize(map_colors)

In [ ]:
IDX = 3220

In [ ]:
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(s3dis_train[IDX].x[:, 3:].numpy())
# pcd.points = o3.utility.Vector3dVector(s3dis_train[IDX].pos.numpy())

pcd.colors = o3.utility.Vector3dVector(s3dis_train[IDX].x[:, :3].numpy())
# colors = np.vstack(v_map_colors(s3dis_train[IDX].y)).T
# pcd.colors = o3.utility.Vector3dVector(colors)

o3.visualization.draw_plotly([pcd])

#### Estimate Mesh for better visualation

http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html

In [ ]:
pcd.estimate_normals()

In [ ]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3.utility.DoubleVector(radii))
o3.visualization.draw_plotly([pcd, rec_mesh])

In [ ]:
print('run Poisson surface reconstruction')
with o3.utility.VerbosityContextManager(
        o3.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=3)
print(mesh)
o3.visualization.draw_plotly([mesh])
